# Create a simple ISA descriptor

This example creates minimal metadata for a single study ISA descriptor with no assay declared. 

It shows how to serialize (write) the ISA Model content to ISA-Tab and ISA-JSON formats.

In [10]:
# If executing the notebooks on `Google Colab`,uncomment the following command 
# and run it to install the required python libraries. Also, make the test datasets available.

# !pip install -r requirements.txt

In [11]:
from isatools.model import (
    Comment,
    Investigation,
    Study,
    StudyFactor,
    FactorValue,
    OntologyAnnotation,
    Material,
    Sample,
    Source,
    Protocol,
    ProtocolParameter,
    ProtocolComponent,
    ParameterValue,
    Process,
    Publication,
    Person,
    Assay,
    DataFile,
    plink
)
import datetime

## Study metadata

In [12]:
investigation = Investigation()
i_comment = Comment(name="i_comment", value="i_value")
investigation.comments.append(i_comment)
study = Study(filename="s_study.txt")
st_comment = Comment(name="st_comment", value="st_value")
study.comments.append(st_comment)
study.identifier = "S1"
study.title = "My Simple ISA Study"
study.description = "We could alternatively use the class constructor's parameters to set some default " \
          "values at the time of creation, however we want to demonstrate how to use the " \
          "object's instance variables to set values."
study.submission_date = str(datetime.datetime.today())
study.public_release_date = str(datetime.datetime.today())
study.sources = [Source(name="source1")]
src_comment = Comment(name="src_comment", value="src_value")
study.sources[0].comments.append(src_comment)
smp_comment = Comment(name="smp_comment", value="smp_value")
study.samples = [Sample(name="sample1")]
study.samples[0].comments.append(smp_comment)


study.protocols = [
    Protocol(name="sample collection",
             components=[
                ProtocolComponent(name="magnetic agitator",
                component_type=OntologyAnnotation(term="device"))]),
    Protocol(
        name="data analysis with Galaxy",
        uri="https://doi.org/10.5464/workflow.cwl",
        protocol_type=OntologyAnnotation(term="data transformation"),
        parameters=[
            ProtocolParameter(parameter_name=OntologyAnnotation(term="genome assembly")),
            ProtocolParameter(parameter_name=OntologyAnnotation(term="cut-off value"))
        ],
    ),
    Protocol(
        name="data visualization with Intermine",
        uri="https://intermine.org/10.5464/network.svg",
        protocol_type=OntologyAnnotation(term="data visualization"),
        comments=[Comment(name="pro_comment", value="pro_value")]
    )
]

study.factors = [
    StudyFactor(name="Factor",factor_type=OntologyAnnotation(term="factor category"))
]


study.publications = [
    Publication(doi="10.12314",pubmed_id="1222322", title="publication title")
    
]

study.contacts = [
    Person(first_name="bob", last_name="hughes", affiliation="WHO", email="bob.hughes@who.else",
           comments=[Comment(name="per_comment", value="per_value")]
    )
]

study.process_sequence = [
    Process(
        executes_protocol=study.protocols[-1],
        inputs=[study.sources[-1]],
        outputs=[study.samples[-1]]
    )
]
investigation.studies = [study]



In [13]:
# Next, we build n Assay object and attach two protocols, extraction and sequencing.

assay = Assay(filename="a_assay.txt")

extraction_protocol = Protocol(name='extraction', protocol_type=OntologyAnnotation(term="material extraction"))
study.protocols.append(extraction_protocol)

sequencing_protocol = Protocol(name='sequencing', protocol_type=OntologyAnnotation(term="material sequencing"))
study.protocols.append(sequencing_protocol)

# To build out assay graphs, we enumereate the samples from the study-level, and for each sample we create an
# extraction process and a sequencing process. The extraction process takes as input a sample material, and produces
# an extract material. The sequencing process takes the extract material and produces a data file. This will
# produce three graphs, from sample material through to data, as follows:
#
# (sample_material-0)->(extraction)->(extract-0)->(sequencing)->(sequenced-data-0)
# (sample_material-1)->(extraction)->(extract-1)->(sequencing)->(sequenced-data-1)
# (sample_material-2)->(extraction)->(extract-2)->(sequencing)->(sequenced-data-2)
#
# Note that the extraction processes and sequencing processes are distinctly separate instances, where the three
# graphs are NOT interconnected.

for i, sample in enumerate(study.samples):

    # create an extraction process that executes the extraction protocol

    extraction_process = Process(executes_protocol=extraction_protocol)

    # extraction process takes as input a sample, and produces an extract material as output

    extraction_process.inputs.append(sample)
    material = Material(name="extract-{}".format(i))
    mat_comment = Comment(name="mat_comment", value="mat_value")
    material.comments.append(mat_comment)
    material.type = "Extract Name"
    extraction_process.outputs.append(material)

    # create a sequencing process that executes the sequencing protocol

    sequencing_process = Process(executes_protocol=sequencing_protocol)
    sequencing_process.name = "assay-name-{}".format(i)
    sequencing_process.inputs.append(extraction_process.outputs[0])

    # Sequencing process usually has an output data file

    datafile = DataFile(filename="sequenced-data-{}".format(i), label="Raw Data File")
    data_comment =  Comment(name="data_comment",value="data_value")
    datafile.comments.append(data_comment)
    sequencing_process.outputs.append(datafile)

    # Ensure Processes are linked forward and backward. plink(from_process, to_process) is a function to set
    # these links for you. It is found in the isatools.model package

    plink(extraction_process, sequencing_process)

    # make sure the extract, data file, and the processes are attached to the assay

    assay.data_files.append(datafile)
    assay.samples.append(sample)
    assay.other_material.append(material)
    assay.process_sequence.append(extraction_process)
    assay.process_sequence.append(sequencing_process)
    assay.measurement_type = OntologyAnnotation(term="genome sequencing")
    assay.technology_type = OntologyAnnotation(term="nucleotide sequencing")

    
study.assays.append(assay)

In [14]:
# Let's see the object :
investigation

isatools.model.Investigation(identifier='', filename='', title='', submission_date='', public_release_date='', ontology_source_references=[], publications=[], contacts=[], studies=[isatools.model.Study(filename='s_study.txt', identifier='S1', title='My Simple ISA Study', description='We could alternatively use the class constructor's parameters to set some default values at the time of creation, however we want to demonstrate how to use the object's instance variables to set values.', submission_date='2021-07-04 11:53:32.752223', public_release_date='2021-07-04 11:53:32.753549', contacts=[isatools.model.Person(last_name='hughes', first_name='bob', mid_initials='', email='bob.hughes@who.else', phone='', fax='', address='', affiliation='WHO', roles=[], comments=[isatools.model.Comment(name='per_comment', value='per_value')])], design_descriptors=[], publications=[isatools.model.Publication(pubmed_id='1222322', doi='10.12314', author_list='', title='publication title', status=None, commen

## Writing to ISA-Tab

In [15]:
from isatools.isatab import dumps
print(dumps(investigation))

/var/folders/5n/rl6lqnks4rqb59pbtpvvntqw0000gr/T/tmp43287hoe/i_investigation.txt
ONTOLOGY SOURCE REFERENCE
Term Source Name
Term Source File
Term Source Version
Term Source Description
INVESTIGATION
Investigation Identifier	
Investigation Title	
Investigation Description	
Investigation Submission Date	
Investigation Public Release Date	
Comment[i_comment]	i_value
INVESTIGATION PUBLICATIONS
Investigation PubMed ID
Investigation Publication DOI
Investigation Publication Author List
Investigation Publication Title
Investigation Publication Status
Investigation Publication Status Term Accession Number
Investigation Publication Status Term Source REF
INVESTIGATION CONTACTS
Investigation Person Last Name
Investigation Person First Name
Investigation Person Mid Initials
Investigation Person Email
Investigation Person Phone
Investigation Person Fax
Investigation Person Address
Investigation Person Affiliation
Investigation Person Roles
Investigation Person Roles Term Accession Number
Investiga

## Writing to ISA-JSON

In [16]:
import json
from isatools.isajson import ISAJSONEncoder
print(json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "comments": [
        {
            "name": "i_comment",
            "value": "i_value"
        }
    ],
    "description": "",
    "identifier": "",
    "ontologySourceReferences": [],
    "people": [],
    "publicReleaseDate": "",
    "publications": [],
    "studies": [
        {
            "assays": [
                {
                    "characteristicCategories": [],
                    "comments": [],
                    "dataFiles": [
                        {
                            "@id": "#data/rawdatafile-5090955616",
                            "comments": [
                                {
                                    "name": "data_comment",
                                    "value": "data_value"
                                }
                            ],
                            "name": "sequenced-data-0",
                            "type": "Raw Data File"
                        }
                    ],
                    "filename": "a_a

In [17]:
import os
from isatools import isatab 
# with open(os.path.join('./BII-I-1/', 'i_investigation.txt')) as fp:
#w ith open(os.path.join('./BII-S-3/', 'i_gilbert.txt')) as fp:
# with open(os.path.join('./BII-S-4/', 'i_investigation.txt')) as fp:
# with open(os.path.join('./BII-S-7/', 'i_matteo.txt')) as fp:
with open(os.path.join('./BII-S-8_FP001RO-isatab-TEST/', 'i_fp001ro-investigation.txt')) as fp:
            ISA = isatab.load(fp)

2021-07-04 11:53:36,527 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,534 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,537 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,539 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,542 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,543 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,545 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for ma

2021-07-04 11:53:36,656 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,659 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,660 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,661 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,662 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,670 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:36,671 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for ma

2021-07-04 11:53:39,619 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:39,620 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:39,620 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:39,621 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:39,622 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:39,622 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:39,623 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for ma

2021-07-04 11:53:59,374 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,375 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,376 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,376 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,377 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,379 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,380 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for ma

2021-07-04 11:53:59,440 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,442 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,443 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,444 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,446 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,449 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,452 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for ma

2021-07-04 11:53:59,511 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,512 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,513 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,514 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,515 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,516 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:53:59,517 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for ma

2021-07-04 11:54:03,285 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:03,286 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:03,287 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:03,288 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:03,289 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:03,290 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:03,291 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for ma

2021-07-04 11:54:03,331 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:03,332 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:03,333 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:03,333 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:09,907 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:09,908 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:09,909 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for ma

2021-07-04 11:54:10,073 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,074 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,075 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,076 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,076 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,077 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,077 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for ma

2021-07-04 11:54:10,114 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,115 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,116 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,116 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,117 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,118 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for material, skipping adding to material object
2021-07-04 11:54:10,120 [WARNING]: isatab.py(create_from_df:5577) >> Duplicate characteristic found for ma

In [18]:
from isatools.isatab import dumps
print(dumps(ISA))

KeyError: 'Extract Name'

## About this notebook

- authors: philippe.rocca-serra@oerc.ox.ac.uk, massimiliano.izzo@oerc.ox.ac.uk
- license: CC-BY 4.0
- support: isatools@googlegroups.com
- issue tracker: https://github.com/ISA-tools/isa-api/issues